# Dependencies and Intallation

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install sacrebleu

In [ ]:
!pip install sentencepiece

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd '/content/drive/MyDrive/SymbolicMath/SymbolicMath'

/content/drive/MyDrive/SymbolicMath/SymbolicMath


In [7]:
from torch.utils.data import DataLoader
from functools import partial
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
from datasets import Dataset
import pandas as pd
from logging import getLogger
import torch
import os
from datasets import load_dataset, load_metric
import csv
import io
import numpy as np
import sympy as sp
import torch
import random
import sys 
from src.utils import AttrDict
from datasets import load_dataset, load_metric
import sentencepiece
from transformers.models.bert.modeling_bert import BertLayer

# Set up and Loading the Data

In [8]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [9]:
params = params = AttrDict({

    # environment parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'prim_fwd',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',
})

In [10]:
from src.envs import build_env
env = build_env(params)         

In [11]:
def read_data(path, number_of_samples):
  with io.open(path, mode='r', encoding='utf-8') as f:
    head = [next(f) for x in range(number_of_samples)]
    lines = [line.rstrip().split('|') for line in head]
    data = [xy.split('\t') for _, xy in lines]
    data = [xy for xy in data if len(xy) == 2]
  return data

path = "prim_fwd.train" 
data = read_data(path, 1000)

In [12]:
print(data[0])
print(data[0][0])

["sub Y' x", 'mul div INT+ 1 INT+ 2 pow x INT+ 2']
sub Y' x


# Pre-Processing the Data
Here we pre-process our data, so that it matches the format in our reference code: [Hugging Face Translation Task Example](https://github.com/huggingface/notebooks/blob/master/examples/translation.ipynb)

## Training Dataset

In [13]:
train_text = []
train_label = []
for i in range(len(data)):
    train_text.append(data[i][0])
    train_label.append(data[i][1])
raw_datasets= [{'en' : train_text[i], 'ro' : train_label[i]} for i in range(len(train_text))]
print(raw_datasets[1])
print(raw_datasets[2]['en'])
print(raw_datasets[2]['ro'])

{'en': "sub Y' mul INT+ 2 x", 'ro': 'pow x INT+ 2'}
sub Y' mul INT+ 2 pow x INT+ 2
mul div INT+ 2 INT+ 3 pow x INT+ 3


In [14]:
raw_datasets_train = {}
for i in range(len(raw_datasets)):
    raw_datasets_train.setdefault('translation',[]).append({'translation' : raw_datasets[i]})
# print(raw_datasets_train['translation'])
print(raw_datasets_train['translation'][1])

{'translation': {'en': "sub Y' mul INT+ 2 x", 'ro': 'pow x INT+ 2'}}


In [15]:
df2 = pd.DataFrame.from_dict(raw_datasets_train['translation']) 
df2

,translation
0,"{'en': 'sub Y' x', 'ro': 'mul div INT+ 1 INT+ ..."
1,"{'en': 'sub Y' mul INT+ 2 x', 'ro': 'pow x INT..."
2,"{'en': 'sub Y' mul INT+ 2 pow x INT+ 2', 'ro':..."
3,"{'en': 'sub Y' add pow x INT+ 2 mul INT+ 2 x',..."
4,"{'en': 'sub Y' mul INT+ 2 pow x INT+ 3', 'ro':..."
...,...
995,"{'en': 'sub Y' mul x exp INT+ 3', 'ro': 'mul d..."
996,{'en': 'sub Y' mul pow x INT- 1 pow ln x INT- ...
997,"{'en': 'sub Y' mul INT+ 3 pow x INT+ 5', 'ro':..."
998,"{'en': 'sub Y' add INT- 1 mul INT+ 3 x', 'ro':..."


In [16]:
train_dataset = Dataset.from_pandas(df2)
print(train_dataset)
print('******************')
print(train_dataset[9])
print('******************')
print(train_dataset[9]['translation'])

Dataset({
    features: ['translation'],
    num_rows: 1000
})
******************
{'translation': {'en': "sub Y' sqrt x", 'ro': 'mul div INT+ 2 INT+ 3 pow x div INT+ 3 INT+ 2'}}
******************
{'en': "sub Y' sqrt x", 'ro': 'mul div INT+ 2 INT+ 3 pow x div INT+ 3 INT+ 2'}


## Validation Dataset

In [42]:
path2 = "prim_fwd.valid" 
data2 = read_data(path2, 100)
valid_text = []
valid_label = []
for i in range(len(data2)):
    valid_text.append(data2[i][0])
    valid_label.append(data2[i][1])
raw_datasets1= [{'en' : valid_text[i], 'ro' : valid_label[i]} for i in range(len(valid_text))]

In [43]:
raw_datasets_vlaid = {}
for i in range(len(raw_datasets1)):
    raw_datasets_vlaid.setdefault('translation',[]).append({'translation' : raw_datasets1[i]})
    
df_valid = pd.DataFrame.from_dict(raw_datasets_vlaid['translation']) 
valid_dataset = Dataset.from_pandas(df_valid)

# Tokenizing the Data

## Their Tokenizer

In [15]:
# model_checkpoint = "Helsinki-NLP/opus-mt-en-ro"  
# metric = load_metric("sacrebleu")
# tokenizer2 = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

# if "mbart" in model_checkpoint:
#     tokenizer.src_lang = "en-XX"
#     tokenizer.tgt_lang = "ro-RO"
# if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
#     prefix = "translate English to Romanian: "
# else:
#     prefix = ""

**Do not run the following cell**

In [ ]:
# tokenizer.encoder
# tokenizer.decoder
# env.word2id['<unk>'] = 91
# env.id2word[91] = '<unk>'
# tokenizer.encoder.clear()
# tokenizer.encoder = env.word2id
# tokenizer.encoder
# tokenizer.decoder.clear()
# tokenizer.decoder = env.id2word
# tokenizer.decoder
# tokenizer.unk_token_id

## Our Tokenizer

In [21]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.pre_tokenizer = Whitespace()

In [22]:
files1 = ['prim_fwd.valid'] 
tokenizer.train(files1, trainer)
# to save the tokenizer
tokenizer.save("tokenizer/gpt2.json")  
# to load the tokenizer
# tokenizer = Tokenizer.from_file("tokenizer/prim_fwd.valid.json")
from transformers import PreTrainedTokenizerFast
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
fast_tokenizer.pad_token = '[PAD]'

In [26]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-ro"  
metric = load_metric("sacrebleu")
# tokenizer2 = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False)

if "mbart" in model_checkpoint:
    fast_tokenizer.src_lang = "en-XX"
    fast_tokenizer.tgt_lang = "ro-RO"
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

# Create the Final Data Set

In [44]:
datasetM = {'train': train_dataset, 'validation': valid_dataset}
datasetM

{'train': Dataset({
     features: ['translation'],
     num_rows: 1000
 }), 'validation': Dataset({
     features: ['translation'],
     num_rows: 100
 })}

In [45]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "ro"
def preprocess_function_new(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = fast_tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with fast_tokenizer.as_target_tokenizer():
        labels = fast_tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [46]:
tokenized_datasets_train = datasetM['train'].map(preprocess_function_new, batched=True)
tokenized_datasets_train

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids', 'translation'],
    num_rows: 1000
})

In [47]:
tokenized_datasets_valid = datasetM['validation'].map(preprocess_function_new, batched=True)
tokenized_datasets_valid

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'token_type_ids', 'translation'],
    num_rows: 100
})

#  Fine-tuning the model

In [36]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [37]:
batch_size = 25
args = Seq2SeqTrainingArguments(
    "test-translation",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [48]:
data_collator = DataCollatorForSeq2Seq(fast_tokenizer, model=model)

In [52]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = fast_tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, fast_tokenizer.pad_token_id)
    decoded_labels = fast_tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != fast_tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [53]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_valid,
    data_collator=data_collator,
    tokenizer=fast_tokenizer,
    compute_metrics=compute_metrics
)

In [54]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,0.920016,3.168600,512.000000


TrainOutput(global_step=40, training_loss=0.988677978515625, metrics={'train_runtime': 349.5712, 'train_samples_per_second': 0.114, 'total_flos': 34200218880000.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 737280, 'train_mem_gpu_alloc_delta': 597979648, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 3901134336})

**Saving and Loading Models**

In [55]:
# torch.save(model, 'models/model1')

In [ ]:
# saved_model = torch.load('models/model1')

**Predicting on one sample**

In [57]:
predicted_labels = trainer.predict(tokenized_datasets_valid)

In [65]:
decoded = fast_tokenizer.decode(predicted_labels.predictions[4], skip_special_tokens=True)
len(decoded)

1651

##  Accuracy

In [63]:
count = 0
for i in range(100):
  
  decoded = fast_tokenizer.decode(predicted_labels.predictions[i], skip_special_tokens=True)
  actual = tokenized_datasets_valid['translation'][i]['ro']
  try:
    actual_s  = convert_to_sympy(actual)
    decoded_s = convert_to_sympy(decoded)
    res = "OK" if simplify(decoded_s - actual_s, seconds=1) == 0 else "NO"
    if res == 'OK':
      count +=1
  except:
    continue
  break

In [64]:
print(count/10, '%')

0.0 %
